# 🧠 Brain Tumor Segmentation Using FFT Filters
## BME 271D Final Project - Ege, Max, Sasha

### Frequency-Domain Analysis for Medical Image Segmentation

**This notebook demonstrates:**
- FFT-based tumor segmentation (High-Pass & Band-Pass filters)
- Comparison with spatial domain methods (Otsu, Blob detection)
- Quantitative evaluation (Dice, IoU, Boundary Accuracy)
- Multi-tier confidence scoring for clinical decision support

In [ ]:
# ========== INSTALLATION & IMPORTS ==========
!pip install -q numpy matplotlib scipy scikit-image pandas
!wget -q https://raw.githubusercontent.com/egeozemek/tumor-segmentation/main/tumor_segmentation.py

import tumor_segmentation as ts
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as PILImage
from google.colab import files
from scipy import ndimage
from scipy.spatial import distance
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects, binary_closing, binary_opening, binary_dilation, disk, erosion
from skimage.segmentation import find_boundaries

print('✅ Setup complete!')

# ========== HELPER FUNCTIONS ==========

def load_image_safe(filepath):
    """Load image and convert to grayscale float [0,1]"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img).astype(np.float64) / 255.0
    return arr

def load_mask_safe(filepath):
    """Load mask and convert to binary [0,1]"""
    img = PILImage.open(filepath).convert('L')
    arr = np.array(img)
    # Convert from 0-255 to 0-1
    return (arr > 127).astype(np.float32)

def remove_peripheral(mask, image, border_frac=0.12):
    """Remove skull and peripheral structures from segmentation."""
    h, w = mask.shape
    border_h, border_w = int(h * border_frac), int(w * border_frac)
    
    # Define internal region (excluding borders)
    internal = np.zeros_like(mask, dtype=bool)
    internal[border_h:-border_h, border_w:-border_w] = True
    
    # Filter regions
    labeled = label(mask)
    cleaned = np.zeros_like(mask, dtype=bool)
    
    for region in regionprops(labeled):
        region_pixels = (labeled == region.label)
        internal_pixels = np.logical_and(region_pixels, internal).sum()
        
        # Region must be >60% internal (not on border)
        if internal_pixels / region.area > 0.6:
            cleaned[region_pixels] = True
    
    return cleaned

# ========== SEGMENTATION METHODS ==========

def otsu_baseline(image):
    """Otsu thresholding baseline method."""
    try:
        thresh = threshold_otsu(image)
        mask = image > thresh
        mask = binary_closing(mask, disk(2))
        mask = remove_small_objects(mask, min_size=100)
        mask = remove_peripheral(mask, image)
        return mask.astype(np.float32)
    except:
        return np.zeros_like(image, dtype=np.float32)

def fft_highpass(image, sensitivity=0.5):
    """FFT High-Pass filter segmentation."""
    try:
        mean_val, std_val = image.mean(), image.std()
        
        # Apply high-pass filter
        hp_img, _, _ = ts.filter_pipeline(image, 'hp', cutoff_radius=25)
        hp_img = (hp_img - hp_img.min()) / (hp_img.max() - hp_img.min() + 1e-8)
        
        # Threshold
        hp_mask = hp_img > np.percentile(hp_img, 87 - sensitivity * 7)
        
        # Must be bright in original
        hp_mask = np.logical_and(hp_mask, image > mean_val + 0.5 * std_val)
        
        # Morphological cleanup
        hp_mask = binary_closing(hp_mask, disk(2))
        hp_mask = remove_small_objects(hp_mask, min_size=80)
        hp_mask = remove_peripheral(hp_mask, image)
        
        return hp_mask.astype(np.float32)
    except:
        return np.zeros_like(image, dtype=np.float32)

def fft_bandpass(image, sensitivity=0.5):
    """FFT Band-Pass filter segmentation."""
    try:
        mean_val, std_val = image.mean(), image.std()
        
        # Apply band-pass filter
        bp_img, _, _ = ts.filter_pipeline(image, 'bp', r1=10, r2=50)
        bp_img = (bp_img - bp_img.min()) / (bp_img.max() - bp_img.min() + 1e-8)
        
        # Threshold
        bp_mask = bp_img > np.percentile(bp_img, 83 - sensitivity * 7)
        
        # Must be bright in original
        bp_mask = np.logical_and(bp_mask, image > mean_val + 0.5 * std_val)
        
        # Morphological cleanup
        bp_mask = binary_closing(bp_mask, disk(2))
        bp_mask = remove_small_objects(bp_mask, min_size=80)
        bp_mask = remove_peripheral(bp_mask, image)
        
        return bp_mask.astype(np.float32)
    except:
        return np.zeros_like(image, dtype=np.float32)

def fft_combined(image, sensitivity=0.5):
    """Combined FFT method (OR of high-pass and band-pass)."""
    hp = fft_highpass(image, sensitivity)
    bp = fft_bandpass(image, sensitivity)
    combined = np.logical_or(hp, bp).astype(np.float32)
    return combined

def blob_detection(image, sensitivity=0.5):
    """Blob detection for bright tumor masses."""
    try:
        h, w = image.shape
        mean_val, std_val = image.mean(), image.std()
        
        # Find very bright regions
        bright_thresh = mean_val + (2.2 - sensitivity * 0.4) * std_val
        percentile_thresh = np.percentile(image, 92 - sensitivity * 5)
        
        blob_mask = np.logical_and(image > bright_thresh, image > percentile_thresh)
        
        # Morphological operations
        blob_mask = binary_opening(blob_mask, disk(2))
        blob_mask = binary_closing(blob_mask, disk(3))
        blob_mask = remove_small_objects(blob_mask, min_size=120)
        blob_mask = remove_peripheral(blob_mask, image)
        
        # Shape filtering
        labeled = label(blob_mask)
        filtered = np.zeros_like(blob_mask)
        
        for region in regionprops(labeled, intensity_image=image):
            # Size constraints
            if region.area < 100 or region.area > h * w * 0.25:
                continue
            
            # Shape constraints
            circularity = 4 * np.pi * region.area / (region.perimeter ** 2 + 1e-8)
            if circularity < 0.12:
                continue
            if region.solidity < 0.65:
                continue
            
            # Intensity constraint
            if region.mean_intensity < mean_val + std_val:
                continue
            
            filtered[labeled == region.label] = True
        
        return filtered.astype(np.float32)
    except:
        return np.zeros_like(image, dtype=np.float32)

def hybrid_combined(image, sensitivity=0.5):
    """Hybrid ensemble combining FFT and blob detection."""
    fft_comb = fft_combined(image, sensitivity)
    blob = blob_detection(image, sensitivity)
    
    # Voting: FFT gets 1.5 votes, Blob gets 1 vote
    vote_map = fft_comb * 1.5 + blob * 1.0
    
    # Need 1.0+ votes (any single method can trigger)
    combined = (vote_map >= (1.0 - sensitivity * 0.1)).astype(np.float32)
    
    # Final cleanup
    combined = binary_closing(combined.astype(bool), disk(2))
    combined = remove_small_objects(combined, min_size=100)
    
    # Shape filter
    labeled = label(combined)
    final = np.zeros_like(combined)
    
    for region in regionprops(labeled):
        if region.eccentricity > 0.96 or region.solidity < 0.65:
            continue
        final[labeled == region.label] = True
    
    return final.astype(np.float32)

# ========== ANALYSIS FUNCTIONS ==========

def get_tumor_center(mask):
    """Get tumor center using largest connected component centroid."""
    if mask.sum() == 0:
        return None
    
    # Find largest connected component
    labeled = label(mask)
    regions = regionprops(labeled)
    
    if len(regions) == 0:
        return None
    
    # Get centroid of largest region
    largest = max(regions, key=lambda x: x.area)
    centroid = largest.centroid  # (row, col)
    
    return (int(centroid[0]), int(centroid[1]))

def calculate_confidence(image, mask):
    """Calculate detection confidence with 3-tier system."""
    h, w = image.shape
    total = h * w
    tumor_px = mask.sum()
    area_pct = (tumor_px / total) * 100
    
    # No detection if too small
    if tumor_px < total * 0.003:  # <0.3%
        return 0.0, 'NONE'
    
    labeled = label(mask)
    regions = regionprops(labeled, intensity_image=image)
    
    if len(regions) == 0:
        return 0.0, 'NONE'
    
    largest = max(regions, key=lambda x: x.area)
    
    # Size score
    if 0.4 <= area_pct <= 12:
        size_score = 1.0
    elif area_pct < 0.4:
        size_score = area_pct / 0.4
    else:
        size_score = max(0, 1 - (area_pct - 12) / 18)
    
    # Shape score
    circularity = 4 * np.pi * largest.area / (largest.perimeter ** 2 + 1e-8)
    shape_score = min(circularity * 2.5, 1.0)
    
    # Contrast score
    tumor_int = image[mask > 0].mean()
    bg_int = image[mask == 0].mean() if (mask == 0).any() else 0
    contrast_score = min(max((tumor_int - bg_int) * 4, 0), 1.0)
    
    # Overall confidence
    confidence = size_score * 0.3 + shape_score * 0.3 + contrast_score * 0.4
    confidence = np.clip(confidence, 0, 1)
    
    # Determine tier
    if confidence >= 0.65:
        tier = 'HIGH'
    elif confidence >= 0.35:
        tier = 'MODERATE'
    elif confidence >= 0.20:
        tier = 'LOW'
    else:
        tier = 'NONE'
    
    return confidence, tier

# ========== QUANTITATIVE METRICS ==========

def dice_coefficient(pred, true):
    """Calculate Dice Similarity Coefficient."""
    pred = (pred > 0.5).astype(np.float32)
    true = (true > 0.5).astype(np.float32)
    
    intersection = np.sum(pred * true)
    pred_sum = np.sum(pred)
    true_sum = np.sum(true)
    
    if pred_sum + true_sum == 0:
        return 1.0  # Both empty
    
    dice = (2.0 * intersection) / (pred_sum + true_sum)
    return dice

def iou_score(pred, true):
    """Calculate Intersection over Union."""
    pred = (pred > 0.5).astype(np.float32)
    true = (true > 0.5).astype(np.float32)
    
    intersection = np.sum(pred * true)
    union = np.sum(np.logical_or(pred, true))
    
    if union == 0:
        return 1.0  # Both empty
    
    iou = intersection / union
    return iou

def boundary_accuracy(pred, true, tolerance=2):
    """Calculate boundary accuracy within tolerance (pixels)."""
    pred = (pred > 0.5).astype(bool)
    true = (true > 0.5).astype(bool)
    
    # Find boundaries
    pred_boundary = find_boundaries(pred, mode='inner')
    true_boundary = find_boundaries(true, mode='inner')
    
    if not pred_boundary.any() or not true_boundary.any():
        return 0.0
    
    # Get boundary coordinates
    pred_coords = np.argwhere(pred_boundary)
    true_coords = np.argwhere(true_boundary)
    
    if len(pred_coords) == 0 or len(true_coords) == 0:
        return 0.0
    
    # For each predicted boundary pixel, find distance to nearest true boundary pixel
    distances = distance.cdist(pred_coords, true_coords, metric='euclidean')
    min_distances = distances.min(axis=1)
    
    # Calculate percentage within tolerance
    within_tolerance = (min_distances <= tolerance).sum()
    accuracy = within_tolerance / len(pred_coords)
    
    return accuracy

def evaluate_all_methods(image, ground_truth, sensitivity=0.5):
    """Evaluate all segmentation methods against ground truth."""
    methods = {
        'Otsu Baseline': otsu_baseline(image),
        'FFT High-Pass': fft_highpass(image, sensitivity),
        'FFT Band-Pass': fft_bandpass(image, sensitivity),
        'FFT Combined': fft_combined(image, sensitivity),
        'Blob Detection': blob_detection(image, sensitivity),
        'Hybrid Combined': hybrid_combined(image, sensitivity)
    }
    
    results = []
    
    for method_name, pred_mask in methods.items():
        dice = dice_coefficient(pred_mask, ground_truth)
        iou = iou_score(pred_mask, ground_truth)
        boundary_acc = boundary_accuracy(pred_mask, ground_truth, tolerance=2)
        
        results.append({
            'Method': method_name,
            'Dice': dice,
            'IoU': iou,
            'Boundary Accuracy': boundary_acc
        })
    
    return pd.DataFrame(results), methods

# Initialize variables
mri_image = None
ground_truth_mask = None

print('\n✅ All functions loaded successfully!')
print('📋 Ready to process images!')

---
## 📤 Step 1: Upload MRI Image (REQUIRED)
Upload your brain MRI scan here.

In [ ]:
# Upload MRI Image
print('📤 Please upload your MRI image...')
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    mri_image = load_image_safe(filename)
    
    print(f'\n✅ MRI Image loaded: {filename}')
    print(f'   Size: {mri_image.shape}')
    print(f'   Range: [{mri_image.min():.3f}, {mri_image.max():.3f}]')
    
    # Display
    plt.figure(figsize=(8, 8))
    plt.imshow(mri_image, cmap='gray')
    plt.title('Uploaded MRI Image', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print('❌ No image uploaded!')

---
## 📤 Step 2: Upload Ground Truth Mask (OPTIONAL)
**Skip this step if you don't have a ground truth mask.**

If you have a mask, upload it here to get quantitative evaluation metrics.

In [ ]:
# Upload Ground Truth Mask (Optional)
print('📤 Upload ground truth mask (or skip if you don\'t have one)...')
uploaded_mask = files.upload()

if uploaded_mask:
    mask_filename = list(uploaded_mask.keys())[0]
    ground_truth_mask = load_mask_safe(mask_filename)
    
    print(f'\n✅ Ground Truth Mask loaded: {mask_filename}')
    print(f'   Size: {ground_truth_mask.shape}')
    print(f'   Unique values: {np.unique(ground_truth_mask)}')
    print(f'   Tumor coverage: {ground_truth_mask.sum() / ground_truth_mask.size * 100:.2f}%')
    
    # Display
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(mri_image, cmap='gray')
    axes[0].set_title('MRI Image', fontsize=12)
    axes[0].axis('off')
    
    axes[1].imshow(ground_truth_mask, cmap='gray')
    axes[1].set_title('Ground Truth Mask', fontsize=12)
    axes[1].axis('off')
    
    axes[2].imshow(mri_image, cmap='gray')
    axes[2].imshow(ground_truth_mask, cmap='Reds', alpha=0.5)
    axes[2].set_title('Overlay', fontsize=12)
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    ground_truth_mask = None
    print('\nℹ️  No mask uploaded - will skip quantitative evaluation.')

---
## 🔬 Step 3: FFT Analysis
Visualize the frequency domain representation of the MRI.

In [ ]:
# FFT Visualization
if mri_image is not None:
    print('🔬 Computing FFT spectrum...')
    F_shift, mag_spectrum = ts.compute_fft_spectrum(mri_image)
    ts.visualize_frequency_spectrum(mri_image, F_shift)
    plt.show()
    print('✅ FFT analysis complete!')
else:
    print('❌ Please upload an MRI image first!')

---
## 🎯 Step 4: Tumor Detection
Run all segmentation methods and visualize results.

In [ ]:
# Run all detection methods
if mri_image is not None:
    print('🎯 Running tumor detection...')
    
    sensitivity = 0.5
    
    # Get all segmentations
    results = {
        'Otsu Baseline': otsu_baseline(mri_image),
        'FFT High-Pass': fft_highpass(mri_image, sensitivity),
        'FFT Band-Pass': fft_bandpass(mri_image, sensitivity),
        'FFT Combined': fft_combined(mri_image, sensitivity),
        'Blob Detection': blob_detection(mri_image, sensitivity),
        'Hybrid Combined': hybrid_combined(mri_image, sensitivity)
    }
    
    # Create visualization
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.ravel()
    
    # Original image
    axes[0].imshow(mri_image, cmap='gray')
    axes[0].set_title('Original MRI', fontsize=13, fontweight='bold')
    axes[0].axis('off')
    
    # Each method
    for idx, (method_name, seg_mask) in enumerate(results.items(), start=1):
        confidence, tier = calculate_confidence(mri_image, seg_mask)
        center = get_tumor_center(seg_mask)
        area_pct = (seg_mask.sum() / seg_mask.size) * 100
        
        axes[idx].imshow(mri_image, cmap='gray')
        axes[idx].imshow(seg_mask, cmap='Reds', alpha=0.5)
        
        # Mark tumor center
        if center is not None:
            r, c = center
            axes[idx].plot(c, r, 'g+', markersize=20, markeredgewidth=3)
        
        # Status text
        if tier == 'HIGH':
            status = f'🔴 HIGH\n{confidence*100:.1f}% | {area_pct:.1f}%'
        elif tier == 'MODERATE':
            status = f'🟡 MODERATE\n{confidence*100:.1f}% | {area_pct:.1f}%'
        elif tier == 'LOW':
            status = f'🟠 LOW\n{confidence*100:.1f}% | {area_pct:.1f}%'
        else:
            status = '🟢 Not detected'
        
        axes[idx].set_title(f'{method_name}\n{status}', fontsize=11, fontweight='bold')
        axes[idx].axis('off')
    
    # Hide last empty subplot
    axes[7].axis('off')
    
    plt.suptitle('Tumor Segmentation - All Methods', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print summary table
    print('\n' + '='*80)
    print(f'{"Method":<20} {"Confidence":<15} {"Tier":<12} {"Area %":<12}')
    print('='*80)
    for method_name, seg_mask in results.items():
        confidence, tier = calculate_confidence(mri_image, seg_mask)
        area_pct = (seg_mask.sum() / seg_mask.size) * 100
        
        conf_str = f'{confidence*100:.1f}%'
        area_str = f'{area_pct:.2f}%' if tier != 'NONE' else '-'
        
        print(f'{method_name:<20} {conf_str:<15} {tier:<12} {area_str:<12}')
    print('='*80)
    
else:
    print('❌ Please upload an MRI image first!')

---
## 📊 Step 5: Quantitative Evaluation (Only if Ground Truth Provided)
Compare all methods using Dice, IoU, and Boundary Accuracy metrics.

In [ ]:
# Quantitative evaluation
if mri_image is not None and ground_truth_mask is not None:
    print('📊 Calculating quantitative metrics...')
    
    # Evaluate all methods
    metrics_df, method_masks = evaluate_all_methods(mri_image, ground_truth_mask, sensitivity=0.5)
    
    # Display table
    print('\n' + '='*80)
    print('QUANTITATIVE EVALUATION RESULTS')
    print('='*80)
    print(metrics_df.to_string(index=False))
    print('='*80)
    
    # Create bar plot comparison
    fig, ax = plt.subplots(1, 1, figsize=(14, 6))
    
    methods = metrics_df['Method'].values
    x = np.arange(len(methods))
    width = 0.25
    
    # Colors: Otsu in gray, FFT methods in blue, Hybrid in red
    colors = ['gray', 'steelblue', 'steelblue', 'steelblue', 'orange', 'darkred']
    
    # Plot bars for each metric
    bars1 = ax.bar(x - width, metrics_df['Dice'], width, label='Dice Coefficient', color=colors, alpha=0.8)
    bars2 = ax.bar(x, metrics_df['IoU'], width, label='IoU Score', color=colors, alpha=0.6)
    bars3 = ax.bar(x + width, metrics_df['Boundary Accuracy'], width, label='Boundary Accuracy', color=colors, alpha=0.4)
    
    # Add value labels on bars
    def add_value_labels(bars):
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}',
                   ha='center', va='bottom', fontsize=8)
    
    add_value_labels(bars1)
    add_value_labels(bars2)
    add_value_labels(bars3)
    
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('Segmentation Performance Comparison\n(Higher is Better)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(methods, rotation=15, ha='right')
    ax.legend(loc='upper left', fontsize=10)
    ax.set_ylim(0, 1.1)
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Analysis
    print('\n📈 ANALYSIS:')
    best_dice = metrics_df.loc[metrics_df['Dice'].idxmax()]
    best_iou = metrics_df.loc[metrics_df['IoU'].idxmax()]
    best_boundary = metrics_df.loc[metrics_df['Boundary Accuracy'].idxmax()]
    
    print(f'   Best Dice: {best_dice["Method"]} ({best_dice["Dice"]:.3f})')
    print(f'   Best IoU: {best_iou["Method"]} ({best_iou["IoU"]:.3f})')
    print(f'   Best Boundary: {best_boundary["Method"]} ({best_boundary["Boundary Accuracy"]:.3f})')
    
    # Compare to Otsu baseline
    otsu_dice = metrics_df[metrics_df['Method'] == 'Otsu Baseline']['Dice'].values[0]
    print(f'\n   Otsu Baseline Dice: {otsu_dice:.3f}')
    
    improvements = []
    for _, row in metrics_df.iterrows():
        if row['Method'] != 'Otsu Baseline' and row['Dice'] > otsu_dice:
            improvement = ((row['Dice'] - otsu_dice) / otsu_dice) * 100
            improvements.append(f"   - {row['Method']}: +{improvement:.1f}% improvement")
    
    if improvements:
        print('\n   Methods outperforming Otsu:')
        for imp in improvements:
            print(imp)
    else:
        print('\n   ⚠️  No method significantly outperformed Otsu baseline')
    
elif mri_image is not None:
    print('\nℹ️  Skipping quantitative evaluation (no ground truth mask provided)')
else:
    print('❌ Please upload an MRI image first!')

---
## 🏥 Step 6: Final Clinical Report
Summary report with the Hybrid Combined method (recommended).

In [ ]:
# Final clinical report
if mri_image is not None and 'results' in dir():
    print('🏥 Generating final clinical report...')
    
    final_mask = results['Hybrid Combined']
    confidence, tier = calculate_confidence(mri_image, final_mask)
    center = get_tumor_center(final_mask)
    area_pct = (final_mask.sum() / final_mask.size) * 100
    
    # Visualization
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Original
    axes[0].imshow(mri_image, cmap='gray')
    axes[0].set_title('Original MRI', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Detection
    axes[1].imshow(mri_image, cmap='gray')
    if tier != 'NONE':
        axes[1].imshow(final_mask, cmap='Reds', alpha=0.5)
        if center is not None:
            r, c = center
            axes[1].plot(c, r, 'g+', markersize=30, markeredgewidth=4)
    axes[1].set_title('Hybrid Detection Result', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Report
    axes[2].axis('off')
    
    if tier == 'HIGH':
        status = '⚠️ TUMOR DETECTED'
        verdict = '🔴 HIGH CONFIDENCE'
        recommendation = 'Urgent follow-up recommended'
    elif tier == 'MODERATE':
        status = '❓ SUSPICIOUS FINDING'
        verdict = '🟡 MODERATE CONFIDENCE'
        recommendation = 'Additional imaging advised'
    elif tier == 'LOW':
        status = '❓ POSSIBLE ABNORMALITY'
        verdict = '🟠 LOW CONFIDENCE'
        recommendation = 'Monitor and re-evaluate'
    else:
        status = '✅ NO TUMOR DETECTED'
        verdict = '🟢 Normal Brain'
        recommendation = 'No immediate action required'
    
    report_text = f"""
    
    {status}
    
    ─────────────────────────
    
    Confidence: {confidence*100:.1f}%
    Affected Area: {area_pct:.2f}%
    
    Method: Hybrid Ensemble
    (FFT + Blob Detection)
    
    ─────────────────────────
    
    {verdict}
    
    {recommendation}
    """
    
    axes[2].text(0.05, 0.5, report_text, fontsize=12, verticalalignment='center',
                fontfamily='monospace', 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    axes[2].set_title('Clinical Report', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print('\n' + '='*70)
    print(f'  {verdict}')
    print('='*70)
    
else:
    print('❌ Please run detection first!')